In [18]:
from pathlib import Path
import pandas as pd

# BASE_DIR = Path(__file__).resolve().parent
BASE_DIR = Path.cwd()

WindowsPath('c:/Users/SEJC94056/Documents/AADT_CodeProject/GAT-Applied/layer_procesing')

In [28]:
ODM_FILE = BASE_DIR / 'data' / 'Mobile Data' / 'odm.csv' 
REAL_NETWORK_MAIN_DIR = BASE_DIR / "/data/geopackages"
NODES_REAL_NETWORK = REAL_NETWORK_MAIN_DIR / "joined_emme_network_nodes_2000_2024.gpkg"
LINKS_REAL_NETWORK = REAL_NETWORK_MAIN_DIR / "joined_emme_network_links_2000_2024.gpkg"

In [35]:
ODM_FILE

WindowsPath('c:/Users/SEJC94056/Documents/AADT_CodeProject/GAT-Applied/layer_procesing/data/Mobile Data/odm.csv')

In [30]:
OD_Data = pd.read_csv(ODM_FILE, sep=";")
OD_Data.head()

,date,hour,origin_zone,destination_zone,people
0,2022-10-03,12,Ryd,Ryd,114
1,2022-10-13,6,Ryd,Ryd,51
2,2022-10-16,7,Ryd,Ryd,7
3,2022-09-27,19,Ryd,Ryd,131
4,2022-09-29,1,Ryd,Ryd,14


In [31]:
OD_Data.columns

Index(['date', 'hour', 'origin_zone', 'destination_zone', 'people'], dtype='object')

In [32]:
# Step 1: Create the datetime column combining 'date' and 'hour'
datetime_col = pd.to_datetime(OD_Data['date'].astype(str) + ' ' + OD_Data['hour'].astype(str) + ':00:00')

# Insert 'datetime' column just after 'hour'
hour_idx = OD_Data.columns.get_loc('hour')
OD_Data.insert(hour_idx + 1, 'datetime', datetime_col)

In [34]:
# Step 2: Create origin_zone_id dictionary and map
origin_dict = {zone: idx for idx, zone in enumerate(OD_Data['origin_zone'].unique())}
origin_zone_id_col = OD_Data['origin_zone'].map(origin_dict)

# Insert 'origin_zone_id' just after 'origin_zone'
origin_zone_idx = OD_Data.columns.get_loc('origin_zone')
OD_Data.insert(origin_zone_idx + 1, 'origin_zone_id', origin_zone_id_col)

# Step 3: Create destination_zone_id dictionary and map
destination_dict = {zone: idx for idx, zone in enumerate(OD_Data['destination_zone'].unique())}
destination_zone_id_col = OD_Data['destination_zone'].map(destination_dict)

# Insert 'destination_zone_id' just after 'destination_zone'
destination_zone_idx = OD_Data.columns.get_loc('destination_zone')
OD_Data.insert(destination_zone_idx + 1, 'destination_zone_id', destination_zone_id_col)


ValueError: cannot insert origin_zone_id, already exists

In [ ]:
OD_Data.head()

In [96]:
import geopandas as gpd

route = r"C:/Users/SEJC94056/Documents/General Layers/Emme_Processed_Layers/emme_links_ready_to_join.gpkg"
emme_links = gpd.read_file(route, layer="emme_links_ready_to_join", force_2d=True)

In [97]:
import csv
import geopandas as gpd
from shapely.geometry import LineString, MultiLineString
from shapely.measurement import frechet_distance
from shapely.ops import linemerge

# -----------------------------
# Funciones auxiliares
# -----------------------------
def ensure_linestring(geom):
    if isinstance(geom, LineString):
        return geom
    elif isinstance(geom, MultiLineString):
        merged = linemerge(geom)
        if isinstance(merged, LineString):
            return merged
        else:
            raise ValueError("No se pudo convertir el MultiLineString a un solo LineString.")
    else:
        raise TypeError("Geometría no soportada.")

def frechet_distance_ignore_direction(geom1, geom2):
    geom1 = ensure_linestring(geom1)
    geom2 = ensure_linestring(geom2)
    dist_forward = frechet_distance(geom1, geom2)
    dist_reverse = frechet_distance(geom1, LineString(list(geom2.coords)[::-1]))
    return (dist_forward, 'normal') if dist_forward <= dist_reverse else (dist_reverse, 'invertida')

# -----------------------------
# Parámetros
# -----------------------------
umbral_porcentual = 0.01  # 1%
output_csv = "comparacion_lineas_por_nodos.csv"

# -----------------------------
# Generar pares únicos de nodos
# -----------------------------
unique_pairs = set(
    frozenset((row['INODE'], row['JNODE'])) for _, row in emme_links.iterrows()
)

# -----------------------------
# Escribir resultados en CSV
# -----------------------------
with open(output_csv, "w", newline='', encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile, delimiter=';')
    
    # Encabezados
    writer.writerow([
        "nodei", "nodej", "frechet_m", "direccion",
        "longitud1_m", "longitud2_m", "longitud_promedio_m",
        "diferencia_relativa_pct", "duplicadas", "atributos_iguales"
    ])
    counter = 0
    for pair in unique_pairs:
        counter+=1
        nodei, nodej = tuple(pair)
        subset = emme_links[
            ((emme_links['INODE'] == nodei) & (emme_links['JNODE'] == nodej)) |
            ((emme_links['INODE'] == nodej) & (emme_links['JNODE'] == nodei))
        ].reset_index(drop=True)

        if len(subset) != 2:
            continue

        try:
            row1, row2 = subset.iloc[0], subset.iloc[1]
            geom1 = ensure_linestring(row1.geometry)
            geom2 = ensure_linestring(row2.geometry)

            # Calcular distancia de Frechet
            distance, direction = frechet_distance_ignore_direction(geom1, geom2)
            len1 = geom1.length
            len2 = geom2.length
            len_prom = (len1 + len2) / 2
            porcentaje = distance / len_prom
            duplicadas = porcentaje <= umbral_porcentual

            # Comparar atributos (excepto INODE, JNODE, geometry)
            excluded = {"ID","INODE", "JNODE", "geometry"}
            common_columns = [col for col in emme_links.columns if col not in excluded]
            if counter == 1:
                print("Atributos comunes:", common_columns)

            atributos_iguales = row1[common_columns].equals(row2[common_columns])

            # Escribir fila
            writer.writerow([
                nodei, nodej,
                round(distance, 3),
                direction,
                round(len1, 2), round(len2, 2),
                round(len_prom, 2),
                round(porcentaje * 100, 2),
                "Sí" if duplicadas else "No",
                "Sí" if atributos_iguales else "No"
            ])
        except Exception as e:
            writer.writerow([nodei, nodej, "Error", str(e)])


Atributos comunes: ['LENGTH', 'TYPE', 'LANES', 'VDF', 'DATA1', 'DATA2', 'DATA3', '@ad_filter', '@adlbs', '@adlbu', '@adpb', '@atk', '@fvkl', '@hast', '@juhas', '@jukap', '@komun', '@lbef', '@vkat', '@vnr', '@vstng', '@vtyp']
